Aluno: **Thiago Brito de Andrade Tenório**

Turma: **Dados 2023.1**

In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [46]:
#Serviço Responsavel por baixar versao mais atual do Chrome que será usada na aplicação (Evita a necessidade de baixar o arquivo chromedriver.exe manualmente)
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service = servico)

#Abre link do Site
navegador.get('https://pt.wikipedia.org/')
#Busca pelo elemento referente ao Buscador e Passa a palavra de busca
navegador.find_element('xpath', '//*[@id="searchInput"]').send_keys('cesar') 

In [48]:
#Clico do Botão "Pesquisar"
navegador.find_element('xpath', '//*[@id="searchform"]/div/button').click() 
#Clico no Link da Instituição
navegador.find_element('xpath', '//*[@id="mw-content-text"]/div[1]/ul[4]/li[2]/a').click()

In [49]:
import requests
from bs4 import BeautifulSoup

link = navegador.current_url
requisicao = requests.get(link)

#Obtem o código html da Página que queremos buscar informações
pagina = BeautifulSoup(requisicao.text, "html.parser")

#Printa a Pagina numa formatação indentada e de Fácil visualização
#print(pagina.prettify())

In [50]:
#Varredura da Tabela de Cursos na Página

table = pagina.find('table', class_='wikitable')

# Lista para armazenar os cursos/programas
cursos_programas = []

# Iterar sobre as linhas da tabela
for linha in table.find_all('tr'):
    
    if linha.find('td'):
        # Obter o texto dentro da célula <td> e adicionar à lista
        curso_programa = linha.find('td').get_text(strip=True)
        cursos_programas.append(curso_programa)

graduacao_01        = cursos_programas[0]
graduacao_02        = cursos_programas[1]
metrado_01          = cursos_programas[2]
mestrado_01         = cursos_programas[3]
doutorado_01        = cursos_programas[4]        

In [51]:
#Buscar dados do quadro superior Direito

table = pagina.find('table', class_='infobox infobox infobox_v2')
# Inicializar variáveis
tipo = None
data_fundacao = None
local_fundacao = None
fundadores = None
fundador_01 = None
fundador_02 = None
fundador_03 = None
ceo = None
coo = None
organizadores = None
numero_funcionarios = None
flag = None

# Iterar sobre as linhas da tabela
for linha in table.find_all('td'):
       
    # Obter o texto dentro da célula <td> e adicionar à lista
    match flag:
        case "Tipo":
            tipo = linha.get_text(strip=True)
        
        case "Sede":
            local_fundacao = linha.get_text(strip=True)  

        case "Fundação":
            data_fundacao = linha.get_text(strip=True)
            fim = data_fundacao.find("(")
            data_fundacao = data_fundacao[:fim]

        case "Fundadores":
            fundadores = linha.get_text(strip=True)

        case "Organização":
            organizadores = linha.get_text(strip=True)
            
        case "Empregados":
            numero_funcionarios = linha.get_text(strip=True)
            numero_funcionarios = numero_funcionarios[:numero_funcionarios.find("(")]
                             
    
    flag = linha.get_text(strip=True)
 
#Obtem Site da Instituição
site = pagina.find("a", class_="external free").text   

# Registrar Informações dos Fundadores:
posicao_um = fundadores.find(",")
posicao_dois = fundadores.find(" e")

if posicao_um != -1:
    fundador_01 = fundadores[:posicao_um]
    fundador_02 = fundadores[posicao_um + 1:posicao_dois +1]
    fundador_03 = fundadores[posicao_dois +2:]
    

# Registrar Informações dos Organizadores:
posicao_ceo = organizadores.find("(CEO)")
# Verificar se "(COO)" foi encontrado
if posicao_ceo != -1:
    # Dividir a string com base na posição de "(COO)"
    ceo = organizadores[:posicao_ceo] + "(CEO)"
    coo = organizadores[posicao_ceo+5:]

In [52]:
#Montagem do Texto Final
 
print(f"""
      O CESAR é um {tipo} fundado em {data_fundacao} por {fundador_01}, {fundador_02}, {fundador_03}
      em {local_fundacao}. Segundo os dados coletados, {ceo} e {coo} são os chefes da organização
      que tem {numero_funcionarios}. O CESAR possui cursos de {graduacao_01}, {graduacao_02}, {metrado_01},
      {mestrado_01}, {doutorado_01}. Para mais detalhes dessa instituição incrível, acesse {site}
""")


      O CESAR é um Instituto de pesquisa sem fins lucrativos fundado em 20 de maiode1996 por Silvio Meira,  Fábio Silva ,  Ismar Kaufman
      em Recife,PE. Segundo os dados coletados, Eduardo Peixoto (CEO) e Karla Godoy (COO) são os chefes da organização
      que tem 600+. O CESAR possui cursos de Ciência da Computação, Design, Engenharia de Software,
      Design de Artefatos Digitais, Engenharia de Software. Para mais detalhes dessa instituição incrível, acesse http://www.cesar.org.br

